In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson 
from scipy.optimize import curve_fit
from pandas.plotting import scatter_matrix
#imports the own created package
import bookie_package as bp

In [79]:
# leagues_short = ['SP1', 'D1', 'E0', 'I1', 'F1', 'N1'] #spanish league, bundesliga, premier league,
# leagues_full = ['la_liga', 'bundesliga', 'premier_league', 'serie_a', 'le_championnat', 'Eredivisie']
# dict_historical_data = {}

# for league, full_name in zip(leagues_short, leagues_full):
#     frames = []
#     for i in range(17, 21):
#         df = pd.read_csv("http://www.football-data.co.uk/mmz4281/"+str(i)+str(i+1)+"/"+league+".csv")
#         print(f'Getting {full_name}-{i}{i+1}...')
#         df = df[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HST', 'AST']] #choose any column you want
#         df = df.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals',
#                                 'HST': 'HomeShotsOnTarget', 'AST':'AwayShotsOnTarget'})
#         df = df.assign(Season=i)
#         frames.append(df)
#     df_historical_data = pd.concat(frames)
#     dict_historical_data[full_name] = df_historical_data


leagues_short = ['D1'] #spanish league, bundesliga, premier league,
leagues_full = ['bundesliga']
dict_historical_data = {}

for league, full_name in zip(leagues_short, leagues_full):
    frames = []
    for i in range(17, 19):
        df = pd.read_csv("http://www.football-data.co.uk/mmz4281/"+str(i)+str(i+1)+"/"+league+".csv")
        print(f'Getting {full_name}-{i}{i+1}...')
        df = df[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HST', 'AST']] #choose any column you want
        df = df.assign(Season=i)
        frames.append(df)
    df_historical_data = pd.concat(frames)
    dict_historical_data[full_name] = df_historical_data

Getting bundesliga-1718...
Getting bundesliga-1819...


In [80]:
for league in leagues_full:
    HTGDIFF = []
    ATGDIFF = []
    for index, row in dict_historical_data[league].iterrows():
        HTGDIFF.append(row['FTHG'] - row['FTAG'])
        ATGDIFF.append(-(row['FTHG'] - row['FTAG']))
    dict_historical_data[league]['HTGDIFF'] = HTGDIFF
    dict_historical_data[league]['ATGDIFF'] = ATGDIFF

dict_historical_data[league]['Date'] = pd.to_datetime(dict_historical_data[league]['Date'], errors='coerce')
dict_historical_data[league]['Day'] = dict_historical_data[league]['Date'].dt.day
dict_historical_data[league]['Month'] = dict_historical_data[league]['Date'].dt.month
dict_historical_data[league]['Year'] = dict_historical_data[league]['Date'].dt.year
dict_historical_data[league] = dict_historical_data[league][['Day','Month','Year','HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HST', 'AST', 'HTGDIFF', 'ATGDIFF']]

dict_historical_data[league]=dict_historical_data[league].sort_values(['Year', 'Month','Day'], ascending=False)
print(dict_historical_data['bundesliga'].head())

     Day  Month  Year       HomeTeam            AwayTeam  FTHG  FTAG  HST  \
296    5     12  2019  Ein Frankfurt               Mainz     0     2    4   
252    4     12  2019       Nurnberg          Schalke 04     1     1    6   
288    5     11  2019       Augsburg              Hertha     3     4    3   
289    5     11  2019       Dortmund  Fortuna Dusseldorf     3     2    5   
290    5     11  2019       Hannover            Freiburg     3     0    6   

     AST  HTGDIFF  ATGDIFF  
296    4       -2        2  
252    2        0        0  
288    8       -1        1  
289    3        1       -1  
290    1        3       -3  


In [59]:
dict_historical_data['bundesliga'].groupby('HomeTeam').count()

,Day,Month,Year,AwayTeam,FTHG,FTAG,HST,AST
HomeTeam,,,,,,,,
Augsburg,34,34,34,34,34,34,34,34
Bayern Munich,34,34,34,34,34,34,34,34
Dortmund,34,34,34,34,34,34,34,34
Ein Frankfurt,34,34,34,34,34,34,34,34
FC Koln,17,17,17,17,17,17,17,17
Fortuna Dusseldorf,17,17,17,17,17,17,17,17
Freiburg,34,34,34,34,34,34,34,34
Hamburg,17,17,17,17,17,17,17,17
Hannover,34,34,34,34,34,34,34,34


In [87]:
dict_data = dict_historical_data['bundesliga']

d_data = bp.averages.avg_goal_diff(dict_data, 'AVGHTGDIFF', 'HomeTeam', 'H')
df_data = bp.averages.from_dict_value_to_df(d_data)
df_data=df_data.sort_values(['Year', 'Month','Day'], ascending=False)


avg_fthg_per_team=bp.averages.avg_goals(df_data, 'AVGFTHG', 'HomeTeam', 'H')
df_data = bp.averages.from_dict_value_to_df(avg_fthg_per_team)
df_data=df_data.sort_values(['Year', 'Month','Day'], ascending=False)


print(df_data[1:10])

     Day  Month  Year    HomeTeam            AwayTeam  FTHG  FTAG  HST  AST  \
252    4     12  2019    Nurnberg          Schalke 04     1     1    6    2   
293    5     11  2019    Nurnberg          M'gladbach     0     4    2    5   
288    5     11  2019    Augsburg              Hertha     3     4    3    8   
289    5     11  2019    Dortmund  Fortuna Dusseldorf     3     2    5    3   
290    5     11  2019    Hannover            Freiburg     3     0    6    1   
291    5     11  2019  Hoffenheim       Werder Bremen     0     1    3    6   
292    5     11  2019  Leverkusen          Schalke 04     1     1    5    3   
294    5     11  2019  RB Leipzig       Bayern Munich     0     0    3    5   
295    5     11  2019   Stuttgart           Wolfsburg     3     0    4    2   

     HTGDIFF  ATGDIFF  AVGHTGDIFF  AVGFTHG  
252        0        0        -0.7      0.7  
293       -4        4        -0.7      0.7  
288       -1        1         0.1      2.3  
289        1       -1        

In [98]:
# HTGDIFF values from the last ten home team games, per past match
team_with_past_HTGDIFF=bp.averages.previous_data(df_data, 'HomeTeam', 'HTGDIFF')
df_team_with_past_HTGDIFF = bp.averages.from_dict_value_to_df(team_with_past_HTGDIFF)

columns_HTGDIFF = ['Day', 'Month', 'Year', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HTGDIFF', 'ATGDIFF', 'AVGHTGDIFF','AVGFTHG','HST', 'AST', 'HTGDIFF_1', 'HTGDIFF_2', 'HTGDIFF_3', 'HTGDIFF_4', 'HTGDIFF_5', 'HTGDIFF_6', 'HTGDIFF_7', 'HTGDIFF_8', 'HTGDIFF_9','HTGDIFF_10']

df_team_with_past_HTGDIFF = df_team_with_past_HTGDIFF.reindex(columns=columns_HTGDIFF)
df_team_with_past_HTGDIFF.fillna(0, inplace=True)

# HST values from the last ten home team games, per past match
team_with_past_HST=bp.averages.previous_data(df_team_with_past_HTGDIFF, 'HomeTeam', 'HST')
df_team_with_past_HST = bp.averages.from_dict_value_to_df(team_with_past_HST)

columns_HST =  ['HST_1', 'HST_2', 'HST_3', 'HST_4', 'HST_5', 'HST_6', 'HST_7', 'HST_8', 'HST_9', 'HST_10']
columns_HTGDIFF_HST = columns_HTGDIFF + columns_HST

df_team_with_past_HST = df_team_with_past_HST.reindex(columns=columns_HTGDIFF_HST)
df_team_with_past_HST.fillna(0, inplace=True)

# FTHG values from the last ten home team games, per past match
team_with_past_FTHG = bp.averages.previous_data(df_team_with_past_HST, 'HomeTeam', 'FTHG')
df_team_with_past_FTHG = bp.averages.from_dict_value_to_df(team_with_past_FTHG)

columns_FTHG = ['FTHG_1', 'FTHG_2', 'FTHG_3', 'FTHG_4', 'FTHG_5', 'FTHG_6', 'FTHG_7', 'FTHG_8', 'FTHG_9', 'FTHG_10']
columns_HTGDIFF_HST_FTHG = columns_HTGDIFF_HST + columns_FTHG

df_team_with_past_FTHG = df_team_with_past_FTHG.reindex(columns=columns_HTGDIFF_HST_FTHG)

# filling na to make data suitable for our model
df_team_with_past_FTHG.fillna(0, inplace=True)
df_result = df_team_with_past_FTHG.copy()

df_result.shape
df_result = df_result.drop(['HomeTeam', 'AwayTeam'], axis = 1)

print(df_result.head())

/Users/guyverchan/Documents/HKU/STAT3622/final_proj/bookie_package/averages.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_with_past_dict[team]['{}_{}'.format(column, i)] = d[i].result
     Day  Month  Year  FTHG  FTAG  HTGDIFF  ATGDIFF  AVGHTGDIFF  AVGFTHG  HST  \
296    5     12  2019     0     2       -2        2         0.0      1.3    4   
272   27      4  2019     0     0        0        0         0.3      1.5    4   
258   14      4  2019     1     3       -2        2         0.6      1.8    4   
241   31      3  2019     3     0        3       -3         1.4      2.4    5   
232   17      3  2019     1     0        1       -1         1.3      2.5    4   

     ...  FTHG_1  FTHG_2  FTHG_3  FTHG_4  FTHG_5  FTHG_6  FTHG_7  FTHG_8 